# Обработка субтитров

## Кодировки

У меня есть около 2000 файлов с субтитрами в формате srt, однако у них у всех разная кодировка - некоторые в UTF-8, некоторые - в CP-1251. Напишем код, чтобы привести все файлы к юникоду.

In [3]:
import os

Пример определения кодировки:

In [12]:
FROM_DIR = './subs'

files = os.listdir(FROM_DIR)
fname = os.path.join(FROM_DIR, files[0])
s = os.popen('chardetect "' + fname + '"').read()  # используем утилиту для командной строки chardetect
enc = s.replace(fname+':', '').strip().split()[0]

with open(fname, 'r', encoding=enc) as f:
    subs = f.read()
    print(subs[:100])

1
00:01:35,500 --> 00:01:36,000
Десять.

2
00:01:36,540 --> 00:01:37,040
Девять.

3
00:01:37,540 -->


А теперь пройдемся по всем файлам:

In [ ]:
FROM_DIR = './subs'
TO_DIR = './converted_subs'

n = 0
for file in os.listdir(FROM_DIR)[290:]:
    fname = os.path.join(FROM_DIR, file)
    # detect encoding
    s = os.popen('chardetect "' + fname + '"').read()
    enc = s.replace(fname+':', '').strip().split()[0]

    try:
        with open(fname, 'r', encoding=enc) as f:
            subs = f.read()
    
        with open(os.path.join(TO_DIR, file), 'w', encoding='utf-8') as f:
            f.write(subs)
        if n % 200 == 0:
            print('>> processed {} files.'.format(n))
        n += 1
    except:
        print('Could not detect encoding of {}'.format(file))

## Реплики и ответы
Теперь нам нужно из всех субтитров собрать все высказывания в виде - Реплика+Ответ. И очистить реплики от мусора.

In [5]:
import re
import random
DIR = './converted_subs'
files = os.listdir(DIR)
reTag = re.compile('<.*?>')
BAD_SYMBOLS = {'’', '≈', '√', '¬', '€', '“', '”', '∆', '„'}

Функция для чистки субтитров:

In [6]:
def cleaned(text):
    utterances = []
    for p in text.split('\n\n'):
        p = p.strip().split('\n')
        if len(p) < 3:
            continue
        p = reTag.sub('', ' '.join(p[2:]).strip()).replace('ƒ', 'д').replace('ќ', 'о')
        if any(i in p for i in BAD_SYMBOLS):
            continue
        if p:
            utterances.append(p)
    return utterances

Проверим, как она работает:

In [7]:
fname = os.path.join(DIR, files[0])
print(fname)
with open(fname, 'r', encoding='utf-8') as f:
    subs = cleaned(f.read().lower())
print(subs[:50])

C:\Users\Admin\Desktop\магистратура\машобуч\converted_subs\.hack Gift OVA [SuYu].srt
['десять.', 'девять.', 'восемь.', 'семь.', 'шесть.', 'пять.', 'четыре.', 'три.', 'два.', 'всем, кто балдеет от серии наших игр:', 'огромное спасибо!', 'что бы выразить благодарность, позвольте мне...', 'поздравления.', 'в знак моей благодарности, я приготовила путешествие к горячим источникам в мире.', 'вы можете неплохо отдохнуть,', 'но путь к источникам спрятан, и найти его не так то просто.', 'на время поисков, можете забыть о целях игры.', 'всем удачи.', 'нежданчик', '[барманк лазурного неба]', 'эх, дождь...', 'дождь прекрасен, как и я, барманк лазурного неба.', '[орка]', 'орка (труп) [фактически медведь].', 'это... невозможно...', 'ааааааааа! смотри, что он вымочил!', 'ооо! блин! что ж это делается?!!', '[кайт, блэкроуз.]', 'он убил его! убийца! пойду-ка я пожалуюсь!', 'подожди минутку. это огромное недоразумение.', 'зачем вдруг мне убивать орку?', 'между вами был спор, так ведь?', 'нет.', 'вы раз

Обработаем все субтитры:

In [8]:
q = []
a = []

n = 0
for file in files:
    fname = os.path.join(DIR, file)
    with open(fname, 'r', encoding='utf-8') as f:
        subs = cleaned(f.read().lower())
    for num in range(1, len(subs)):
        q.append(subs[num-1])
        a.append(subs[num])
    if n % 200 == 0:
        print('>> processed {} files.'.format(n))
    n += 1
    
print(len(q), len(a))

>> processed 0 files.
>> processed 200 files.
>> processed 400 files.
>> processed 600 files.
>> processed 800 files.
>> processed 1000 files.
>> processed 1200 files.
>> processed 1400 files.
>> processed 1600 files.
>> processed 1800 files.
>> processed 2000 files.
682561 682561


Распечатаем для проверки несколько случайных реплик:

In [9]:
print(random.sample(q, 100))

['мы позаботимся об остальном.', 'надо бы денежку подготовить.', 'круто!', 'конечно.', '[языки пламени достигали 20 метров] [томое хотару - 9 лет и профессор соичи томое] весь научно-исследовательский персонал погиб...', '[это козо]', 'а что?', 'что? значит, все эти продажи коту насмарку?', '[бинбо, 17 лет]', '[указания по изменению планов захвата города текст: штаб квартира акросс музыка и аранжировка: тохио масуда исполнитель: кей]', 'я сейчас прикончу эту девчонку!', 'её используют против демона, которого иначе не победить.', 'доброе утро мисс ребекка!', 'как там дело с этим саругами?', 'хватит уже о харуко-сан!!', 'что это вы двое делаете?', 'like a...date?', 'ты был прав. мы ничего не сможем здесь для нее сделать.', 'что...', 'и вдруг ты говоришь "я проголодался, дай чего-нибудь пожевать" - а затем начинаешь ковыряться в моей сумке!', 'инуяша...', 'самом сильном воине из легенд - сейлор мун.', 'как мило!', 'алло, айти?', 'прошу простить, но в михаре есть люди,  которые когда-то мн

## Подготовка данных

In [10]:
def prepare_seq2seq_files(questions, answers, path='', TESTSET_SIZE = 30000):
    
    # open files
    train_enc = open(path + 'train.enc','w', encoding='utf-8')
    train_dec = open(path + 'train.dec','w', encoding='utf-8')
    test_enc  = open(path + 'test.enc', 'w', encoding='utf-8')
    test_dec  = open(path + 'test.dec', 'w', encoding='utf-8')

    # choose 30,000 (TESTSET_SIZE) items to put into testset
    test_ids = random.sample([i for i in range(len(questions))],TESTSET_SIZE)

    for i in range(len(questions)):
        try:
            if i in test_ids:
                test_enc.write(questions[i]+'\n')
                test_dec.write(answers[i]+ '\n' )
            else:
                train_enc.write(questions[i]+'\n')
                train_dec.write(answers[i]+ '\n' )
            if i%10000 == 0:
                print ('>> written {} lines'.format(i))
        except UnicodeEncodeError:
            print(questions[i], answers[i])

    # close files
    train_enc.close()
    train_dec.close()
    test_enc.close()
    test_dec.close()

In [ ]:
prepare_seq2seq_files(q, a, TESTSET_SIZE = 80000, 
                      path=r"./tensorflow_chatbot/working_dir/")